# Setup



In [ ]:
# Set this up to support Vector search [previously pip install azure-search-documents==11.4.0b8]
#pip install azure-search-documents --pre  

#pip install openai

In [14]:
import os
import requests
import time
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt  

from azure.core.credentials import AzureKeyCredential

import openai

In [9]:
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  

In [18]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Azure OpenAI Resources
openai_api_base = config_details['OPENAI_API_BASE']
openai_api_key = config_details["OPENAI_API_KEY"]
openai_api_version = config_details['OPENAI_API_VERSION']

chat_model_deployment = config_details['GPT_MODEL']
embeddings_model_deployment = config_details['EMBEDDING_MODEL']

# Azure Cognitive Search Resources
AZSEARCH_KEY = config_details['AZSEARCH_KEY']
AZSEARCH_ENDPOINT= config_details['AZSEARCH_ENDPOINT']
AZSEARCH_INDEX_NAME = config_details['AZSEARCH_INDEX_NAME']
AZSEARCH_API_VERSION = config_details['AZSEARCH_API_VERSION']

In [19]:
openai.api_base = openai_api_base
openai.api_key = openai_api_key
openai.api_version = openai_api_version
openai.api_type = "azure"

In [21]:
#Establish for connectivity to Azure Cognitive Search throughout
credential = AzureKeyCredential(AZSEARCH_KEY)

In [22]:
# Function to Generate Embeddings using OpenAI Ada 002

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for use query
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine=embeddings_model_deployment)
    embeddings = response['data'][0]['embedding']
    return embeddings


## Query Cognitive Search

Uses pure vector search to find most relevant documents and format for use in GPT Prompt.

In [78]:
# Pure Vector Search
query = "tools for video analysis"  
  
search_client = SearchClient(AZSEARCH_ENDPOINT, AZSEARCH_INDEX_NAME, credential=credential)
vector = Vector(value=generate_embeddings(query), k=10, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vectors= [vector],
    select=["title", "content", "category"],
)  
  
docs_list = []
  
for result in results:  
    # print(f"Title: {result['title']}")  
    # print(f"Score: {result['@search.score']}")  
    # print(f"Text: {result['content']}")  
    # print(f"Category: {result['category']}\n")  
    docs_list.append(f"{result['content']} [{result['title']}] ^{result['@search.score']}")

docs_list

['Azure Media Services is a cloud-based media processing and delivery platform that enables you to build, deploy, and scale video applications. It provides features like encoding, streaming, content protection, and video indexing. Media Services supports various input and output formats, such as MP4, MPEG-DASH, and HLS. You can use Media Services to deliver live and on-demand video, build video-on-demand platforms, and analyze your video content. It also integrates with other Azure services, such as Azure Content Delivery Network and Azure Storage. [Azure Media Services] ^0.8114831',
 'Azure Stream Analytics is a real-time data stream processing service that enables you to analyze and process high volumes of fast-moving data. It supports various data sources, such as Azure Event Hubs, Azure IoT Hub, and Azure Blob Storage. Stream Analytics provides features like windowing, time-based aggregations, and user-defined functions. You can use Stream Analytics to build real-time dashboards, d

# Setup ChatGPT Prompt

Create the message (prompt) to send to ChatGPT.  With the [Chat Completion API](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions#working-with-the-chat-completion-api) there are distinct sections of the prompt that are sent to the API in the form of an array of dictionaries with associated roles: system, user, and assistant. 

In [75]:
prompt = []

#setting up the personality and clear instructions for the assistant
system_message = "You're an Azure Support Specialist helping customers and partners understand the services available. \
You are given a question and a context. You need to answer the question using only the context. \
Be friendly and conversational, don't sound like a computer! \
The context contains information about a service, with the title of the source document enclosed by [] and the relevancy search score follows a ^ \
Format the information from the three most relevant services into a bulleted list. \
If there are no good matches tell the user you cannot find anything, but they can ask again. \
Always include the document title the information was sourced from at the end of each suggestion enclosed in []. \
Do not show the scores to the user."

prompt.append({"role":"system", "content": system_message})
prompt

# adding some special instructions and an example
prompt.append({"role":"user", "content": f"Print the document titles at the end of the suggestion enclosed in []"})
prompt.append({"role": "assistant", "content": f"Azure Stream Analytics is a real-time data stream processing service that enables you to analyze and process high volumes of fast-moving data [Azure Stream Analytics]."})

In [80]:
# Add the user's search question with the specific context (documents from Azure Search).
prompt.append({"role":"user", "content":f"Question: {query} <context> {docs_list} </context>"})

completion = openai.ChatCompletion.create(
engine=chat_model_deployment,
messages=prompt,
temperature=0.7,  #adjust as needed to make more or less random/deterministic
)

suggestions = completion.choices[0].message['content']
print(suggestions)


Based on the information provided, here are three Azure services that can be used for video analysis:

- Azure Media Services: This cloud-based platform enables you to build, deploy, and scale video applications. It provides features like encoding, streaming, content protection, and video indexing. You can use Media Services to deliver live and on-demand video, build video-on-demand platforms, and analyze your video content. [Azure Media Services]

- Azure Stream Analytics: This real-time data stream processing service allows you to analyze and process high volumes of fast-moving data. It supports various data sources such as Azure Event Hubs, Azure IoT Hub, and Azure Blob Storage. Stream Analytics offers features like windowing, time-based aggregations, and user-defined functions. It can be used to build real-time dashboards, detect anomalies, and generate alerts. [Azure Stream Analytics]

- Azure Time Series Insights: This fully managed analytics, storage, and visualization service i